In [11]:
import mitsuba as mi
import drjit as dr 
import matplotlib.pyplot as plt
mi.set_variant("cuda_ad_rgb")

# Surface Reconstruction based on Photometric Stereo

input:
- image $I\in\mathbb{R}^{256\times256\times3}$
- Light Source Information
  - LEDs position $P\in\mathbb{R}^{24\times3}$
  - LEDs intensity $E\in\mathbb{R}^{24\times3}$
  - LEDs direction $D\in\mathbb{R}^{24\times3}$
- Camera Information
  - Camera resolution
  - Camera Position
  - Camera fov
- Initial Estimation of Surface $Z_0\in\mathbb{R}^{256\times256}$

output:
- Estimation of Surface $Z\in\mathbb{R}^{256\times256}$

## Get Image $I$

In [ ]:
## get image I ##
bmp = mi.Bitmap('../scenes/img/normal-10-10.png')

I_t = mi.TensorXf(bmp)/256

I_R = I_t[:,:,0]
I_G = I_t[:,:,1]
I_B = I_t[:,:,2]

f, ax = plt.subplots(2,2)
ax[0,0].imshow(I_t); ax[0,0].axis('off');ax[0,0].set_title("$I$")
ax[0,1].imshow(I_R,cmap='gray');ax[0,1].axis('off');ax[0,1].set_title("$I_R$");
ax[1,0].imshow(I_G,cmap='gray');ax[1,0].axis('off');ax[1,0].set_title("$I_G$");
ax[1,1].imshow(I_B,cmap='gray');ax[1,1].axis('off');ax[1,1].set_title("$I_B$");

## Set LED Configuration

In [13]:
import utils.led_ring

N_LED = 24
LED0_pos = mi.Point3f(34.41,0,-12.5)
LED_ring = utils.led_ring.LED_params(N_LED, LED0_pos)
P_LED = LED_ring.P
D_LED = LED_ring.D

## get LED intensity ##
A = 0.4
B = 0.5
# LED_ring.set_sine_intensity(A,B)
LED_ring.set_sine_intensity(A,B)


## Set Camera Configuration

In [14]:
Intrinsics = mi.Matrix3f([[422.3949, 0,313.5413],[0,422.4885,233.1770],[0,0,1]])
Z_camera = -44
image_res = (640,480)
cam_conner1 = dr.inverse(Intrinsics)@mi.Point3f(0,0,1)*-Z_camera
cam_conner2 = dr.inverse(Intrinsics)@mi.Point3f(image_res[0]-1,image_res[1]-1,1)*-Z_camera

# ### camera origin          ###
cam_origin = mi.Point3f(0,0,Z_camera)
cam_range = ((cam_conner1[0,0],cam_conner1[1,0]),(cam_conner2[0,0],cam_conner2[1,0]))
cam_fov= (cam_conner2[0,0]-cam_conner1[0,0],cam_conner2[1,0]-cam_conner1[1,0])
from utils import camera 
cam = camera.Camera(cam_origin, image_res,cam_range)

## Calculate Surface Normal and Surface Gradient

In [15]:
from utils import iristac

sensor = iristac.sensor(cam,LED_ring)
normal = sensor.get_surface_normal(I_t)
# normal = sensor.si.n
### Calculate gradient ###
import numpy as np
grad_x = np.reshape(-normal[0]/normal[2],(image_res[1],image_res[0]))
grad_y = np.reshape(-normal[1]/normal[2],(image_res[1],image_res[0]))

In [ ]:
I = sensor.get_tactile_img()
plt.imshow(I)
plt.show()

## Surface Reconstruction

In [17]:
### Ground truth ###
scene_ref = mi.load_file('../scenes/iristac_ply.xml')
si_ref = cam.shot(scene_ref)
Z_ref = si_ref.p[2]

In [ ]:
from utils.depth_from_grad import frankotchellappa, fast_poisson
from decimal import Decimal

N_Pixel = image_res[0]*image_res[1]
"""Frankot-Chellappa depth-from-gradient algorithm."""
Z = frankotchellappa(grad_x,grad_y)
Z = Z*np.sqrt(cam_fov[0]*cam_fov[1]/N_Pixel)
error_rms = np.mean(np.sqrt((np.ravel(Z)-Z_ref)**2))
error_std = np.std((np.ravel(Z)-Z_ref)**2)
print("Frankot-Chellappa reconstruction error:\n\
rms error:  \t {0:.2E}\n\
std  error:  \t {1:.2E}\n\
".format(Decimal(error_rms),Decimal(error_std))
)


"""Fast Poisson Solver."""
Z = fast_poisson(grad_x,grad_y)
Z = Z*np.sqrt(cam_fov[0]*cam_fov[1]/N_Pixel)
error_rms = np.mean(np.sqrt((np.ravel(Z)-Z_ref)**2))
error_std = np.std((np.ravel(Z)-Z_ref)**2)
print("Fast Poisson Solver reconstruction error:\n\
rms error:  \t {0:.2E}\n\
std  error:  \t {1:.2E}\n\
".format(Decimal(error_rms),Decimal(error_std))
)

## Simulation of Tactile Image with Estimated Depth

In [ ]:
### Get Tactile Image with Estimation Depth ###
Z_t = mi.TensorXf(np.ravel(Z),(cam.image_res[1],cam.image_res[0]))
sensor.set_Z0(Z_t)
I_compute = sensor.get_tactile_img()

f, ax = plt.subplots(1,2)

ax[0].imshow(I_compute);ax[0].axis('off');  ax[0].set_title('Reconstruction Result')
ax[1].imshow(I_t);      ax[1].axis('off');  ax[1].set_title('Refrence Image')
plt.show()